In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [2]:
url = "https://raw.githubusercontent.com/atikagondal/Lab-2023-DAVE3625/main/Lab3/data/flight.csv"

df = pd.read_csv(url)
df.head(3)

,Unnamed: 0,datetime_val,dep_time,arr_time,sched_arr_time,flight,origin,dest
0,0,2013-1-1 05:40:00,2013-1-1 05:42:00,2013-1-1 09:23:00,2013-1-1 08:50:00,1141,JFK,MIA
1,1,2013-1-1 06:00:00,2013-1-1 05:54:00,2013-1-1 08:12:00,2013-1-1 08:37:00,461,LGA,ATL
2,2,2013-1-1 05:58:00,2013-1-1 05:54:00,2013-1-1 07:40:00,2013-1-1 07:28:00,1696,EWR,ORD


In [3]:
# Konverter kolonnen dep_time til datotype datetime, og ignorer feilmeldinger ('coerce')
df['dep_time'] = pd.to_datetime(df['dep_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Konverter kolonnen datetime_val til datotype datetime
df['datetime_val'] = pd.to_datetime(df['datetime_val'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Gjør det samme for arr_time og sched_arr_time
df['arr_time'] = pd.to_datetime(df['arr_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')
df['sched_arr_time'] = pd.to_datetime(df['sched_arr_time'], format='%Y-%m-%d %H:%M:%S', errors='coerce')


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121427 entries, 0 to 121426
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   Unnamed: 0      121427 non-null  int64         
 1   datetime_val    121427 non-null  datetime64[ns]
 2   dep_time        121421 non-null  datetime64[ns]
 3   arr_time        121379 non-null  datetime64[ns]
 4   sched_arr_time  121427 non-null  datetime64[ns]
 5   flight          121427 non-null  int64         
 6   origin          121427 non-null  object        
 7   dest            121427 non-null  object        
dtypes: datetime64[ns](4), int64(2), object(2)
memory usage: 7.4+ MB


In [5]:
# Beregn air_time som er tiden mellom dep_time og arr_time
df['air_time'] = df['arr_time'] - df['dep_time']


#Delay (forsinkelsen mellom arr_time og sched_arr_time):
df['Delay'] = df['arr_time'] - df['sched_arr_time']


df.head()

,Unnamed: 0,datetime_val,dep_time,arr_time,sched_arr_time,flight,origin,dest,air_time,Delay
0,0,2013-01-01 05:40:00,2013-01-01 05:42:00,2013-01-01 09:23:00,2013-01-01 08:50:00,1141,JFK,MIA,0 days 03:41:00,0 days 00:33:00
1,1,2013-01-01 06:00:00,2013-01-01 05:54:00,2013-01-01 08:12:00,2013-01-01 08:37:00,461,LGA,ATL,0 days 02:18:00,-1 days +23:35:00
2,2,2013-01-01 05:58:00,2013-01-01 05:54:00,2013-01-01 07:40:00,2013-01-01 07:28:00,1696,EWR,ORD,0 days 01:46:00,0 days 00:12:00
3,3,2013-01-01 06:00:00,2013-01-01 05:55:00,2013-01-01 09:13:00,2013-01-01 08:54:00,507,EWR,FLL,0 days 03:18:00,0 days 00:19:00
4,4,2013-01-01 06:00:00,2013-01-01 05:57:00,2013-01-01 08:38:00,2013-01-01 08:46:00,79,JFK,MCO,0 days 02:41:00,-1 days +23:52:00


In [ ]:
# For hver rad i dataframe df
for index, row in df.iterrows():
    # Sjekk om air_time er negativ (dager er mindre enn 0)
    if row['air_time'].days < 0:
        # Beregn den korrigerte air_time ved å trekke den negative air_time fra 24 timer
        corrected_air_time = datetime.timedelta(hours=24) - (row['air_time'] + datetime.timedelta(days=abs(row['air_time'].days)))
        # Oppdater verdien i dataframe med den korrigerte verdien
        df.loc[index, 'air_time'] = corrected_air_time







In [ ]:
#Delay as a percent of air time (forsinkelse som prosent av air_time):
df['Delay_as_percent_of_air_time'] = (df['Delay'].dt.total_seconds() / df['air_time'].dt.total_seconds()) * 100
df.head()

In [ ]:
# Beregn forsinkelsen som en prosentandel av air_time
df['percent_delay'] = (df['Delay'].dt.total_seconds() / df['air_time'].dt.total_seconds()) * 100


In [ ]:
df.head()

In [ ]:
df["percent_delay"].plot.box()

In [ ]:
minrow= df['percent_delay'].argmin()

In [ ]:
df.iloc[minrow]

In [ ]:
df["percent_delay"].plot.box()


In [ ]:
df["percent_delay"].describe()

In [ ]:
from pandas.api.types import is_numeric_dtype

def remove_outlier(df):
    low = 0.05
    high = 0.95
    quant_df = df.quantile([low, high])
    if is_numeric_dtype(df):
        df = df[(df >= quant_df.loc[low]) & (df <= quant_df.loc[high])]
    return df

# Bruk funksjonen på percent_delay-kolonnen
df["percent_delay"] = remove_outlier(df["percent_delay"])


In [ ]:
import matplotlib.pyplot as plt

# Plot boksdiagram for percent_delay-kolonnen
plt.figure(figsize=(8, 6))
df["percent_delay"].plot.box()
plt.title("Box Plot of Percent Delay")
plt.show()

# Bruk describe for å se statistiske opplysninger
print(df["percent_delay"].describe())